In [1]:
%load_ext lab_black

# Honey Bee Computer Vision Example

This example shows how to use python scripts from open-source repos to train a yolov5 model, and evaluate the results.

The example trains a model to detect Honey Bees using the V7 version of the https://github.com/ultralytics/yolov5. The purpose of the example is to explain how to train a model using Kubeflow, the YOLOV5 repo and sample data. 

Accessed classes are:
* Bees (workers or foragers)
* Bees carrying pollen
* Drones
* Queens


Datasets for training of object detection models are usually large, and take a long time to train. They usually contain many pre-augmented images. For our purposes, we've included a sample of training data to use for this exercise. It will not produce a good model, but it will train quickly, and allow experimentation with the pipeline.

We also included a pretrained model using a much larger version of this dataset with 500+ epochs (This takes many hours to train).

The dataset was sampled from here: https://universe.roboflow.com/matt-nudi/honey-bee-detection-model-zgjnb (creative commons license, but requires collection of an email to download).

The assumption is that this notebook has a data volume mounted (you can set this up when you create the notebook). The PVC should have been created with an access mode of ReadWriteMany. 
This allows other pods to mount the volume.

Also assumed is that the notebook is running with a base image that has the yolov5 packages installed. You can see how the image was created by looking at the file in kubeflow-ppc64le-examples/object-detection-yolov5/Notebook Container Image Source/DockerFile.

The data set will be extracted to the data volume in these next few cells. The data will be loaded into the pipeline via a volume, rather than a download. This simulates a use case where the training data has been pre loaded on a volume in the kubernetes cluster, and is large enough where a download is expensive.

The volume name is defined in this next cell, as is the path to the extracted Roboflow data set for the bees. To keep things simple, the mount point is the same for both the notebook server, and also for the containers that mount the volume.

In [2]:
VOLUME_CLAIM_NAME = "yolov5-work"
MOUNT_POINT = "/home/jovyan/vol-1"
BEE_DATA_SET_PATH = f"{MOUNT_POINT}/bee_dataset"

In [3]:
!mkdir -p {BEE_DATA_SET_PATH}
!tar -xf data/dataset.tar.gz -C {BEE_DATA_SET_PATH} --strip-components 1

## Copy YOLOV5 files to (notebook) persistent storage
The base image has cloned the yolov5 repo into the root directory tree. (`/yolov5`). 

When the yolov5 scripts are executed, they will produce output in the same directory tree.

The root directory is part of the containers R/W layer. That means that changes there are _NOT_ persisted. The root directory is also not accessible from the navigation bar of our notebook server.

We'll copy the yolov5 files into our home directory, and that way we can run commands interactively and examine results with the file browser. Because the home directory has been mounted to a volume, the changes there will not be lost if the notebook server is shutdown.

This copy is for interactive work in the notebook, the pipeline does not use this.

In [4]:
%%bash

if [ ! -d /home/jovyan/yolov5 ] 
then 
   cp -R /yolov5 /home/jovyan/yolov5 
else 
   echo "The directory is already copied"
fi

The directory is already copied


## Imports and constants

If you've build your own yolov5 container, you should change the BASE_IMAGE variable to your own image here.

In [5]:
import kfp
from kfp import dsl
from kfp.components import InputPath, OutputPath
from kubernetes.client.models import (
    V1Volume,
    V1VolumeMount,
    V1PersistentVolumeClaimVolumeSource,
)
from kfp.dsl import PipelineConf, data_passing_methods
import os
from typing import Optional

BASE_IMAGE = "quay.io/ntlawrence/yolov5:pt1.12.1-yolo7.0-v1.1"
COMPONENT_CATALOG_FOLDER = f"{os.getenv('HOME')}/components"

## Load Data component
The first component in the pipeline copies the data from an input path to an output parameter.

Essentially this moves the data from the volume into the pipeline. A common problem when using PVCs is that paths to data in a PVC (strings), are not interchangable with pipeline inputs and outputs (InputPath and OutputPath). Components like this are needed to transform data on a PVC into something that can be used by existing pipeline components that expect InputPath and OutputPath params.

In [6]:
def load_from_url(
    source: str,
    dest: OutputPath(str),
):
    import os
    import shutil
    from urllib.request import urlretrieve
    from urllib.parse import urlparse

    # Make target directories if needed
    parent_dirs = os.path.dirname(dest)
    if not os.path.exists(parent_dirs):
        os.makedirs(parent_dirs)

    # Option to use an empty file to indicate no weights
    if not source:
        with open(dest, "w") as _:
            pass

    source_details = urlparse(source)

    if source_details.scheme == "file":
        if os.path.isdir(source_details.path):
            shutil.copytree(source_details.path, dest)
        else:
            shutil.copyfile(source_details.path, dest)
    elif source_details.scheme in ("http", "https", "ftp", "ftps"):
        urlretrieve(source, filename=dest)
    else:
        raise ValueError(f"source does not use a supported url scheme")


load_from_url_comp = kfp.components.create_component_from_func(
    load_from_url, base_image=BASE_IMAGE
)

## Copy data from one path to another
This component is a helper to move data from one volume path to another.

This component allows us to copy pipeline artifacts to a mounted PVC for use outside of the pipeline.

In [7]:
def copy_data(source: str, dest: str):
    import os
    import shutil

    # Make target directories if needed
    parent_dirs = os.path.basename(dest)
    if not os.path.exists(parent_dirs):
        os.makedirs(parent_dirs)

    if os.path.isdir(source):
        shutil.copytree(source, dest)
    else:
        shutil.copyfile(source, dest)


copy_data_comp = kfp.components.create_component_from_func(
    copy_data, base_image=BASE_IMAGE
)

## Train Model component

Run the python train.py CLI to train the model

Performance could be improved by using distributed training. This may be investigated and included in the example in the future.

Integration with tensorboard is also an area that might be investigated and included in the future.

The trained model and results.csv are reported as output parameters.

In [8]:
def train_model(
    model: OutputPath(str),
    results: OutputPath(str),
    model_cfg: InputPath(str),
    initial_weights: InputPath(str),
    epochs: int,
):
    import subprocess
    import pathlib
    from ruamel.yaml import YAML
    import os
    import shutil

    # Option to pass an empty file to train from scratch
    weights = initial_weights if os.path.getsize(initial_weights) > 0 else ""

    subprocess.run("find /dataset -print", shell=True)
    subprocess.run(
        f"python train.py --img 640 --batch -1 --noplots --epochs {epochs} --cache /home/jovyan/cache --cfg={model_cfg} "
        f"--data /dataset/data.yaml --weights {weights} --workers=0 --device=0 --optimizer=Adam",
        check=True,
        cwd="/yolov5",
        shell=True,
    )

    os.makedirs(os.path.dirname(model), exist_ok=True)
    os.makedirs(os.path.dirname(results), exist_ok=True)

    shutil.copyfile("/yolov5/runs/train/exp/weights/best.pt", model)
    shutil.copyfile("/yolov5/runs/train/exp/results.csv", results)


train_model_comp = kfp.components.create_component_from_func(
    train_model, base_image=BASE_IMAGE
)

## Component to convert the model to ONNX

The export tool supports quantizing, the model, and so we've added that option as a parameter to the component. The ability to quantize is important for models that will be used for inferencing on edge devices. Many edge devices do not have the resources to evaluate deep neural networks, and and smaller model makes it possible to run inferencing in those environments.

In [9]:
def convert_model_to_onnx(
    model: InputPath(str),
    model_format: str,
    onnx_model: OutputPath(str),
    quantize: str = "",
):
    import subprocess
    import shutil
    import os

    # export.py uses the file name to determine the type of model
    # mode is an input path where the name is generated by kubeflow
    # We need to control the name that is used...
    named_model = f"/tmp/{os.path.basename(model)}.{model_format}"
    os.symlink(model, named_model)

    quantize_param = f"--{quantize}" if quantize else ""

    subprocess.run(
        f"python export.py --img 640 --include=onnx  {quantize_param} "
        f"--data /dataset/data.yaml --weights {named_model} --device=cpu ",
        check=True,
        cwd="/yolov5",
        shell=True,
    )

    shutil.copyfile(f"/tmp/{os.path.basename(model)}.onnx", onnx_model)


convert_model_to_onnx_comp = kfp.components.create_component_from_func(
    convert_model_to_onnx, base_image=BASE_IMAGE
)

## Model evaluation component

This component can be used for both the original and onnx models. This allows comparisons between the ONNX model (which might be quantized), and the original model.

In [10]:
def evaluate_model(
    results: OutputPath(str),
    model: InputPath(str),
    model_format: str = "onnx",  # onnx, pt, tf ....
    conf_thres: float = 0.001,
    iou_thres: float = 0.6,
    max_det: int = 300,
):
    import subprocess
    import os
    import torch
    from ruamel.yaml import YAML
    import pathlib
    import shutil

    print(f"The size of the model is {os.path.getsize(model)}")

    if model_format == "onnx" and not torch.cuda.is_available():
        # the base image is built with an onnxruntime for GPU
        # This should work for both CPU and GPU, but val.py
        # does it's own checking for CPU onnxruntime only
        # Since that's not installed and not on pypl for ppc64le,
        # The script won't work unless we change up the version
        subprocess.run(
            "mamba install -c rocketce onnxruntime=1.13.1=hea80eff_cpu_py39_pb3.19_1 -y",
            check=True,
            shell=True,
        )

    # valy.py uses the file name to determine the type of model
    # mode is an input path where the name is generated by kubeflow
    # We need to control the name that is used...
    named_model = f"/tmp/{os.path.basename(model)}.{model_format}"
    os.symlink(model, named_model)

    subprocess.run(
        f"python val.py --weights {named_model} --data /dataset/data.yaml --img 640 "
        f"--conf-thres {conf_thres} --iou-thres {iou_thres} --max-det {max_det} --workers=0 ",
        check=True,
        shell=True,
        cwd="/yolov5",
    )

    os.makedirs(os.path.dirname(results), exist_ok=True)
    shutil.copytree("/yolov5/runs/val/exp", results)


evaluate_model_comp = kfp.components.create_component_from_func(
    evaluate_model, base_image=BASE_IMAGE
)

## Write artifact to PVC

Converts data from a pipeline parameter to a file stored on a PVC.

In [41]:
def write_artifact_to_path(source: InputPath(str), dest: str(str)):
    import os
    import shutil

    parent_dirs = os.path.dirname(dest)
    os.makedirs(parent_dirs, exist_ok=True)

    if os.path.isdir(source):
        shutil.copytree(source, dest)
    else:
        shutil.copyfile(source, dest)


write_artifact_to_path_comp = kfp.components.create_component_from_func(
    write_artifact_to_path, base_image=BASE_IMAGE
)

## Upload the ONNX model

The upload component is the same component that is shared with other examples. It loads the ONNX model into MinIO.

In [42]:
UPLOAD_MODEL_COMPONENT = (
    f"{COMPONENT_CATALOG_FOLDER}/model-building/upload-model/component.yaml"
)

upload_model_comp = kfp.components.load_component_from_file(UPLOAD_MODEL_COMPONENT)

## Deploy Model Component

Deploys the model to a NVIDIA Triton inference service

In [43]:
DEPLOY_MODEL_COMPONENT = f"{os.getenv('HOME')}/kubeflow-ppc64le-examples/deploy_triton_inference_service_component/deploy_triton_inference_service_component.yaml"
deploy_model_comp = kfp.components.load_component_from_file(DEPLOY_MODEL_COMPONENT)

## Pipeline Definition

This defines the pipeline, and the pipeline's paramters. We'll compile this pipeline into a YAML file that we can upload, and access through the UI.

In [44]:
BLACKBOARD_RESOURCE_NAME = "ml-blackboard"

In [58]:
@dsl.pipeline(name="bee-yolov5")
def bee_yolov5(
    data_vol_pvc_name: str,
    data_vol_subpath: str,
    epochs: int = 750,
    model_config_url: str = "https://github.com/ultralytics/yolov5/raw/v7.0/models/yolov5s.yaml",
    initial_weights_url: str = "https://github.com/ultralytics/yolov5/releases/download/v7.0/yolov5s.pt",
    quantize_onnx: str = "int8",
    minio_url="minio-service.kubeflow:9000",
    model_version: int = 1,
    dataset_size: str = "4Gi",
    artifact_vol_pvc_name: str = "",
    artifact_vol_subpath: str = "",
):
    def mount_volume(task, pvc_name, mount_path, volume_subpath, read_only=False):
        task.add_volume(
            V1Volume(
                name=pvc_name,
                persistent_volume_claim=V1PersistentVolumeClaimVolumeSource(pvc_name),
            )
        )

        task.add_volume_mount(
            V1VolumeMount(
                name=pvc_name,
                mount_path=mount_path,
                sub_path=volume_subpath,
                read_only=read_only,
            )
        )

    create_blackboard = dsl.VolumeOp(
        name="Create Artefacts Blackboard",
        resource_name=BLACKBOARD_RESOURCE_NAME,
        modes=dsl.VOLUME_MODE_RWO,
        size="4Gi",
        set_owner_reference=True,
    )

    DATASET_VOLUME_NAME = "dataset-pvc"
    # Pipeline automatically adds {{workflow.name} prefix when creating
    # the resource!
    DATASET_VOLUME = "{{workflow.name}}-" + DATASET_VOLUME_NAME
    create_dataset_volume = dsl.VolumeOp(
        name=f"Create PVC for dataset",
        resource_name=DATASET_VOLUME_NAME,
        modes=dsl.VOLUME_MODE_RWM,
        size=dataset_size,
        set_owner_reference=True,
    )

    # Load config  and data Tasks
    model_config_task = load_from_url_comp(model_config_url)
    model_config_task.after(create_blackboard)
    model_config_task.set_display_name("Load model Config")

    initial_weights_task = load_from_url_comp(initial_weights_url)
    initial_weights_task.after(create_blackboard)
    initial_weights_task.set_display_name("Load initial weights")

    copy_dataset_task = copy_data_comp("/src-vol", "/dest-vol/dataset")
    mount_volume(
        copy_dataset_task,
        data_vol_pvc_name,
        "/src-vol",
        data_vol_subpath,
        read_only=True,
    )
    mount_volume(copy_dataset_task, DATASET_VOLUME, "/dest-vol", "")
    copy_dataset_task.after(create_dataset_volume)
    copy_dataset_task.after(create_blackboard)
    copy_dataset_task.set_display_name("Copy dataset to Pipeline owed PVC")

    # Train Model
    train_model_task = train_model_comp(
        model_cfg=model_config_task.outputs["dest"],
        initial_weights=initial_weights_task.outputs["dest"],
        epochs=epochs,
    )
    train_model_task.set_gpu_limit(1)
    train_model_task.set_memory_limit("30G")
    mount_volume(train_model_task, DATASET_VOLUME, "/dataset", "dataset")
    train_model_task.after(copy_dataset_task)

    # convert to ONNX
    convert_model_to_onnx_task = convert_model_to_onnx_comp(
        model=train_model_task.outputs["model"],
        model_format="pt",
        quantize=quantize_onnx,
    )

    # Evaluate
    evaluate_onnx_model_task = evaluate_model_comp(
        model=convert_model_to_onnx_task.outputs["onnx_model"],
    )
    evaluate_onnx_model_task.set_display_name("Evaluate ONNX Model")
    mount_volume(evaluate_onnx_model_task, DATASET_VOLUME, "/dataset", "dataset")

    evaluate_pt_model_task = evaluate_model_comp(
        model=train_model_task.outputs["model"],
        model_format="pt",
    )
    evaluate_pt_model_task.set_display_name("Evaluate Torch Model")
    mount_volume(evaluate_pt_model_task, DATASET_VOLUME, "/dataset", "dataset")

    # Upload ONNX model
    upload_model_task = upload_model_comp(
        convert_model_to_onnx_task.outputs["onnx_model"],
        minio_url=minio_url,
        export_bucket="{{workflow.namespace}}-bee",
        model_format="onnx",
        model_name="bee",
        model_version=model_version,
    )

    # Deploy Inference Service
    deploy_model_task = deploy_model_comp(
        name="bee",
        rm_existing=True,
        storage_uri="s3://{{workflow.namespace}}-bee/onnx",
        minio_url=minio_url,
        predictor_protocol="v2",
    )
    deploy_model_task.after(upload_model_task)

    ##### Write evaluation results
    with dsl.Condition(artifact_vol_pvc_name != "", name="store_onnx_metrics"):
        ## Save ONNX Metrics
        write_onnx_artifact_to_path_task = write_artifact_to_path_comp(
            evaluate_onnx_model_task.outputs["results"],
            "/mnt/{{workflow.name}}/onnx-metrics",
        )
        mount_volume(
            write_onnx_artifact_to_path_task,
            artifact_vol_pvc_name,
            "/mnt",
            artifact_vol_subpath,
        )
        write_onnx_artifact_to_path_task.set_display_name("Save ONNX metrics (PVC)")

        ## Save Torch Metrics
        write_torch_artifact_to_path_task = write_artifact_to_path_comp(
            evaluate_pt_model_task.outputs["results"],
            "/mnt/{{workflow.name}}/pt-metrics",
        )
        mount_volume(
            write_torch_artifact_to_path_task,
            artifact_vol_pvc_name,
            "/mnt",
            artifact_vol_subpath,
        )
        write_torch_artifact_to_path_task.set_display_name("Save Torch results (PVC)")

        ## Save Torch Model
        write_torch_model_artifact_to_path_task = write_artifact_to_path_comp(
            train_model_task.outputs["model"],
            "/mnt/{{workflow.name}}/model.pt",
        )
        mount_volume(
            write_torch_model_artifact_to_path_task,
            artifact_vol_pvc_name,
            "/mnt",
            artifact_vol_subpath,
        )
        write_torch_model_artifact_to_path_task.set_display_name(
            "Save Torch model (PVC)"
        )

        ## Save ONNX Model
        write_onnx_model_artifact_to_path_task = write_artifact_to_path_comp(
            convert_model_to_onnx_task.outputs["onnx_model"],
            "/mnt/{{workflow.name}}/model.onnx",
        )
        mount_volume(
            write_onnx_model_artifact_to_path_task,
            artifact_vol_pvc_name,
            "/mnt",
            artifact_vol_subpath,
        )
        write_onnx_model_artifact_to_path_task.set_display_name("Save ONNX model (PVC)")

## Compile Pipeline with configuration options

Uses a PVC for passing data between components.

In [59]:
# See: https://www.kubeflow.org/docs/components/pipelines/overview/caching/#managing-caching-staleness
def disable_cache_transformer(op):
    if isinstance(op, dsl.ContainerOp):
        op.execution_options.caching_strategy.max_cache_staleness = "P0D"
    else:
        op.add_pod_annotation(
            name="pipelines.kubeflow.org/max_cache_staleness", value="P0D"
        )
    return op


pipeline_conf = PipelineConf()
pipeline_conf.add_op_transformer(disable_cache_transformer)

pipeline_conf.data_passing_method = data_passing_methods.KubernetesVolume(
    volume=V1Volume(
        name=BLACKBOARD_RESOURCE_NAME,
        persistent_volume_claim=V1PersistentVolumeClaimVolumeSource(
            "{{workflow.name}}-" + BLACKBOARD_RESOURCE_NAME
        ),
    ),
    path_prefix=f"{BLACKBOARD_RESOURCE_NAME}/",
)

In [60]:
PIPELINE_NAME = "Bee detector pipeline"

kfp.compiler.Compiler().compile(
    pipeline_func=bee_yolov5,
    package_path=f"{PIPELINE_NAME}.yaml",
    pipeline_conf=pipeline_conf,
)

## Upload pipeline programmatically

The YAML file generated by the previous compile can be used to create a pipeline through the UI.
* Download the file to your workstation
* Click "Pipelines" from the side bar
* Press the "upload pipeline" button, and upload the YAML.

After adding the pipline, you can run the pipeline from the UI without looking at the pipeline code. This allows an inexperienced user to run the pipeline with specific parameters, without the compelxity of Kubeflow componet code or K8S Awareness.

These next few cells upload and run the pipeline programmatically, so that the example is "automated".

In [61]:
def delete_pipeline(pipeline_name: str):
    """Delete's a pipeline with the specified name"""

    client = kfp.Client()
    existing_pipelines = client.list_pipelines(page_size=999).pipelines
    matches = (
        [ep.id for ep in existing_pipelines if ep.name == pipeline_name]
        if existing_pipelines
        else []
    )
    for id in matches:
        client.delete_pipeline(id)


def get_experiment_id(experiment_name: str) -> str:
    """Returns the id for the experiment, creating the experiment if needed"""
    client = kfp.Client()
    existing_experiments = client.list_experiments(page_size=999).experiments
    matches = (
        [ex.id for ex in existing_experiments if ex.name == experiment_name]
        if existing_experiments
        else []
    )

    if matches:
        return matches[0]

    exp = client.create_experiment(experiment_name)
    return exp.id

In [62]:
# Pipeline names need to be unique, so before we upload,
# check for and delete any pipeline with the same name
delete_pipeline(PIPELINE_NAME)

In [63]:
# upload the pipeline
client = kfp.Client()
uploaded_pipeline = client.upload_pipeline(f"{PIPELINE_NAME}.yaml", PIPELINE_NAME)

## Run the pipeline with parameters

This is equivalent to running the pipeline from the pipelines view in the UI. Since a pipeline run needs to be part of an experiment, this code will create the experiment if it does not exist.

When we run this manually, we'll use initial weights that have been trained on a much larger superset of this data, and with many more epochs. That will give us results that can be demoed with a fast training cycle.

In [64]:
pipeline_params = {
    "data_vol_pvc_name": VOLUME_CLAIM_NAME,
    "data_vol_subpath": "bee_dataset",
    "initial_weights_url": "https://github.com/ntl-ibm/kubeflow-ppc64le-examples/raw/main/object-detection-yolov5/data/model.onnx",
    "epochs": "15",
    "artifact_vol_pvc_name": VOLUME_CLAIM_NAME,
    "artifact_vol_subpath": "runs",
}

run = client.run_pipeline(
    experiment_id=get_experiment_id("bee-exp"),
    job_name="bees",
    pipeline_id=uploaded_pipeline.id,
    params=pipeline_params,
)

## Waits for the pipeline to complete 

* waits for up to 20 Min
* Checks the results

In [ ]:
TWENTY_MIN = 20 * 60
result = client.wait_for_run_completion(run.id, timeout=TWENTY_MIN)
{
    "status": result.run.status,
    "error": result.run.error,
    "time": str(result.run.finished_at - result.run.created_at),
}

## Inference Example

Since the pipeline deploys an inference service, you will see the inference service in the 'Models' list. 

The URL for the service can used for the weights. The detect tool knows how to talk to NVIDIA triton infrence servers, and the model has already been deployed to this service by the pipeline.

In [ ]:
IMAGE = f"{BEE_DATA_SET_PATH}/test/images/2016-03-15-04-09-38-1024x673_jpg.rf.aa352f3bd2a105dd7ff560e0ab42ae69.jpg"

In [ ]:
!python /home/jovyan/yolov5/detect.py --weights=http://bee.kubeflow-ntl.svc.cluster.local/v2/models/bee/infer --data=$BEE_DATA_SET_PATH/data.yaml --source=$IMAGE --conf-thres=.7 --iou-thres=.2 --max-det=500

Because we have an ONNX model in our repo that was trained over many epocs, we can use that model, instead of the InferenceService


In [ ]:
!python /home/jovyan/vol-1/yolov5/detect.py --weights=/home/jovyan/kubeflow-ppc64le-examples/object-detection-yolov5/data/model.onnx --data=/home/jovyan/vol-1/bee_data/data.yaml --source=$IMAGE --conf-thres=.7 --iou-thres=.2 --max-det=500